In [1]:
## Tunes depth profiles of Korolev to each other using dynamic time warping algorithm
## Depth profiles should be .csv files located in ./korolev
## Plots tunings and computes cross-correlation
## Tests XC value against XC's from tunings to random profiles

In [2]:
#Import packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
from radarfuncs import *
from DTWfuncs import *
from lmfit.models import SkewedGaussianModel

In [ ]:
#Import csv files
csv_import = glob.glob("korolev/*zoom.csv")

column_names = ['depth', 'value']
csv_list = []
x_list = []
y_list = []
fit_list = []

#load data and put into lists
for i in csv_import:
    csv = pd.read_csv(i, names=column_names, skiprows=2) #skips headers
    csv_list.append(csv)
    
    x = csv.depth.to_list()
    y = csv.value.to_list()
    
    peak = y.index(max(y[0:50])) #finds surface return
    
    x = x[peak:]
    y = y[peak:]
    
    #convert pixels to meters
    x = p2m_waterice(x)
    
    x_list.append(x)
    y_list.append(y)
    
    ##compute line to fit synthetic data to
    # Skewed Gaussian fit
    model = SkewedGaussianModel()
    params = model.make_params(amplitude=(max(y)), center=min(x)+20, sigma=20, gamma=1.5)
    result = model.fit(y, params, x=x)
    fit = result.best_fit +np.mean(y[-150:]) # prevents fit from damping out all of the noise on right-handed side
    
    fit_list.append(fit)

#Use DTW to tune each profile to the other profiles
#plot results, produce XC

#Run MonteCarlo simulation to assess quality of tunings
#A strong match is when real XC is nigher than 90% of all XCs from tunings to random profiles
